In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-u-s-election-sentiment-on-x/val.csv
/kaggle/input/2024-u-s-election-sentiment-on-x/train.csv
/kaggle/input/2024-u-s-election-sentiment-on-x/test.csv


# Environment Setup

In [2]:
# Installing libraries
!pip install seaborn numpy pandas matplotlib nltk

In [3]:
# pandas for maniplating, cleaning, and exploring data
import pandas as pd
# numerical computing in Python. arrays and performance optimization
import numpy as np
# plotting
import matplotlib.pyplot as plt
# built ontop of plt, heatmaps, boxplots, distribution plots
import seaborn as sns
# basic natural language processing. tokenizations
# stopwords, stemming, etc
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

## Load Datasets

In [4]:
train = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/train.csv')
test = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/test.csv')
val = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/val.csv')

## Display first few rows of each dataset

### Training Data
Training data from the 2024 US Presidential election, sourcced from X (fromerly Twitter). It includes post text, candidate information, party affiliation, sentitment labels, retweet counts, and like. This data is ideal for training machine learning models to analyze public sentiment and trends.

In [5]:
print('Train Dataset:', train.head())

Train Dataset:    tweet_id     user_handle            timestamp  \
0         1        @user123  2024-11-03 08:45:00   
1         2    @politicsFan  2024-11-03 09:15:23   
2         3  @greenAdvocate  2024-11-03 10:05:45   
3         4     @indieVoice  2024-11-03 11:20:10   
4         5   @libertyLover  2024-11-03 12:35:55   

                                          tweet_text       candidate  \
0  Excited to see Kamala Harris leading the Democ...   Kamala Harris   
1  Donald Trump's policies are the best for our e...    Donald Trump   
2  Jill Stein's environmental plans are exactly w...      Jill Stein   
3  Robert Kennedy offers a fresh perspective outs...  Robert Kennedy   
4  Chase Oliver's libertarian stance promotes tru...    Chase Oliver   

               party  retweets  likes sentiment  
0   Democratic Party       120    450  positive  
1   Republican Party        85    300  positive  
2        Green Party        60    200  positive  
3        Independent        40    150  

### Testing Data
This file provides testing data from the 2024 US Presidential election. It contains post text, candidate details, party affilication, and engagement metrics. This dataset is used to evaluate the performance of models trains on the training data.

In [6]:
print('\n Test Dataset:', test.head())


 Test Dataset:    tweet_id       user_handle            timestamp  \
0       501         @user1068  2025-02-11 08:30:10   
1       502     @politicsNew2  2025-02-11 09:15:20   
2       503  @greenVisionary2  2025-02-11 10:05:35   
3       504    @indieWatcher2  2025-02-11 11:25:40   
4       505      @libertyFan4  2025-02-11 12:40:50   

                                          tweet_text       candidate  \
0  Kamala Harris's new policy on education reform...   Kamala Harris   
1  Donald Trump's latest speech on the economy wa...    Donald Trump   
2  Jill Stein's environmental policies are a step...      Jill Stein   
3  Robert Kennedy's independent run brings fresh ...  Robert Kennedy   
4  Chase Oliver's stance on gun rights is unwaver...    Chase Oliver   

               party  retweets  likes sentiment  
0   Democratic Party        85    320  positive  
1   Republican Party       100    450  positive  
2        Green Party        60    200  positive  
3        Independent      

### Validation Data
This file contains validation data for the 2024 US Presidential election. It includes post content, candidates, party affiliation, and sentiment labels. Use this file to fine-tune and validate model accuracy and performance. 

In [7]:
print('Validation Dataset:', val.head())

Validation Dataset:    tweet_id       user_handle            timestamp  \
0       551         @user1077  2025-02-21 08:30:10   
1       552     @politicsNew3  2025-02-21 09:15:20   
2       553  @greenVisionary3  2025-02-21 10:05:35   
3       554    @indieWatcher3  2025-02-21 11:25:40   
4       555      @libertyFan5  2025-02-21 12:40:50   

                                          tweet_text       candidate  \
0  Kamala Harris's new policy on education reform...   Kamala Harris   
1  Donald Trump's latest speech on the economy wa...    Donald Trump   
2  Jill Stein's environmental policies are a step...      Jill Stein   
3  Robert Kennedy's independent run brings fresh ...  Robert Kennedy   
4  Chase Oliver's stance on gun rights is unwaver...    Chase Oliver   

               party  retweets  likes sentiment  
0   Democratic Party        85    320  positive  
1   Republican Party       100    450  positive  
2        Green Party        60    200  positive  
3        Independent  

# Data Exploration

Dataset overview for all 3 files

### `pandas.info()`
`.info()` is a pandas method that "prints a concise summary oof a DataFrame.
This method provides a concise summary about a DataFrame including data types, non-null values, and memeory usage for each column. <br>


In [8]:
print('Train Dataset Info:')
train.info()

Train Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     500 non-null    int64 
 1   user_handle  500 non-null    object
 2   timestamp    500 non-null    object
 3   tweet_text   500 non-null    object
 4   candidate    500 non-null    object
 5   party        500 non-null    object
 6   retweets     500 non-null    int64 
 7   likes        500 non-null    int64 
 8   sentiment    500 non-null    object
dtypes: int64(3), object(6)
memory usage: 35.3+ KB


In [9]:
print('Test Dataset Info: ')
test.info()

Test Dataset Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     50 non-null     int64 
 1   user_handle  50 non-null     object
 2   timestamp    50 non-null     object
 3   tweet_text   50 non-null     object
 4   candidate    50 non-null     object
 5   party        50 non-null     object
 6   retweets     50 non-null     int64 
 7   likes        50 non-null     int64 
 8   sentiment    50 non-null     object
dtypes: int64(3), object(6)
memory usage: 3.6+ KB


In [10]:
print('Validation Dataset Info: ')
val.info()

Validation Dataset Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     50 non-null     int64 
 1   user_handle  50 non-null     object
 2   timestamp    50 non-null     object
 3   tweet_text   50 non-null     object
 4   candidate    50 non-null     object
 5   party        50 non-null     object
 6   retweets     50 non-null     int64 
 7   likes        50 non-null     int64 
 8   sentiment    50 non-null     object
dtypes: int64(3), object(6)
memory usage: 3.6+ KB
